# Use Case Where A Model Is Tested With Signals Collected From A Different Place of The Body
(e.g wrist vs belt, ankle vs wrist, ankle vs belt)

## Loading Dependencies

In [6]:
# Reference: https://www.kaggle.com/xpuspus/ml-dl-human-activity-recognition

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Any results you write to the current directory are saved as output.
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot as plt
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

import keras
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix as cf
from keras.utils.np_utils import to_categorical
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.utils import vis_utils
from keras.utils import plot_model
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix as cf
# from keras.utils.np_utils import to_categorical
from sklearn.decomposition import PCA
from sklearn.preprocessing import label_binarize
#from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.model_selection import cross_val_score as cv_score
import time
from sklearn.metrics import roc_curve as roc
from sklearn.metrics import auc as auc

## Loading Training Dataset

## Readme File Of FUSMPA-2014 ##

Important note: In section 3 (data collection) in our paper, the term running is mistakenly used for the jogging. So please only consider the jogging activity.

In the data collection experiments, we collected data for seven physical activities. These are walking, sitting, standing, jogging, biking, walking upstairs and walking downstairs, which are mainly used in the related studies and they are the basic motion activities in daily life. There were ten participants involved in our data collection experiment who performed each of these activities for 3-4 minutes. All ten participants were male, between the ages of 25 and 30. The experiments were carried out indoors in one of the university buildings, except biking. For walking, and jogging, the department corridor was used. For walking upstairs and downstairs, a 5-floor building with stairs was used. Each of these participants was equipped with five smartphones on five body positions: 
1.	One in their right jean’s pocket. 
2.	One in their left jean’s pocket.
3.	One on belt position towards the right leg using a belt clipper.
4.	One on the right upper arm. 
5.	One on the right wrist. 
The first three positions are commonly used by people carrying smartphones. The fourth position is usually used when activities like jogging are performed. However, we used this position for all activities to see its role on the performance.  A smart-watch was simulated with the fifth position as smart-watches are coming into the market these days.  For these experiments, we used Samsung Galaxy SII (i9100) smartphones. 
  The orientation of the smartphones was portrait for the upper arm, wrist, and two pockets, and landscape for the belt position. The data was recorded for all five positions at the same time for each activity and it was collected at a rate of 50 samples per second. This sampling rate (50 samples per second) is enough to recognize human physical activities, as we show in our previous study . Moreover, in the state of the art, frequencies lower than 50 samples per second have been shown to be sufficient for activity recognition.
For data collection, we adapted our own data collection app from our previous study by adding the linear acceleration sensor. The data was collected for an accelerometer, a gyroscope, a magnetometer, and a linear acceleration sensor. 

Each excel file contains data for each participant's seven physical activities for all five positions. 

Notation in these files: 

Accelerometer ( Ax = x-axis, Ay = y-axis, Az= Z-aixs)   
Linear Acceleration Sensor ( Lx = x-axis, Ly = y axis, Lz= Z-aixs) 
Gyroscope ( Gx = x-axis, Gy = y-axis, Gz= Z-aixs) 
Magnetometer ( Mx = x-axis, My = y-axis, Mz= Z-aixs)

In [34]:
train_data = pd.read_csv('../pre-processing/valid-dataset/Participant_1_belt.csv')
train_data.append(pd.read_csv('../pre-processing/valid-dataset/Participant_2_belt.csv'))
train_data.append(pd.read_csv('../pre-processing/valid-dataset/Participant_3_belt.csv'))
train_data.append(pd.read_csv('../pre-processing/valid-dataset/Participant_4_belt.csv'))
train_data.append(pd.read_csv('../pre-processing/valid-dataset/Participant_5_belt.csv'))
train_data.append(pd.read_csv('../pre-processing/valid-dataset/Participant_6_belt.csv'))
train_data.append(pd.read_csv('../pre-processing/valid-dataset/Participant_7_belt.csv'))
train_data.append(pd.read_csv('../pre-processing/valid-dataset/Participant_8_belt.csv'))
train_data.append(pd.read_csv('../pre-processing/valid-dataset/Participant_9_belt.csv'))

,ax,ay,az,lx,ly,lz,gx,gy,gz,mx,my,mz,label
0,5.1213,-2.19290,-0.708260,-4.53520,-0.676290,0.081656,0.436160,0.163710,0.198230,-21.90,7.14,5.34,walking
1,5.6116,-2.12480,-1.266700,-4.04080,-0.559740,-0.523690,0.322840,0.431270,0.211970,-21.60,7.50,5.64,walking
2,6.0611,-1.47100,-2.478900,-3.59080,0.157950,-1.880200,0.204030,0.728460,0.164930,-21.12,8.40,6.12,walking
3,7.7091,-0.77636,-3.541300,-1.94500,0.857960,-2.995300,0.098960,0.943790,0.064752,-20.94,8.70,6.30,walking
4,11.2100,-0.98067,-1.675300,1.54300,0.635280,-1.334200,0.183260,0.812150,-0.033292,-20.70,9.30,6.42,walking
5,12.6120,-1.51190,0.980670,2.94000,0.083211,1.244200,0.588570,0.520460,-0.251370,-20.64,9.66,6.30,walking
6,15.8540,-5.85670,2.233700,5.12710,-2.534300,3.376300,0.646600,0.351250,-0.259920,-20.70,10.02,6.18,walking
7,17.2430,-4.60370,-10.120000,7.57480,-2.971000,-9.964100,0.048869,0.315510,0.356130,-20.76,10.62,5.82,walking
8,17.0250,-2.51980,-10.815000,7.36530,-0.864190,-10.480000,-0.409890,0.004887,0.536650,-20.88,10.92,5.76,walking
9,13.2250,1.19860,-3.568500,3.57760,2.893500,-3.101200,-1.141100,-1.471600,0.151190,-21.00,11.16,5.76,walking


In [176]:
X_train = train_data.drop('label', axis = 1)
X_train.head(2)

,ax,ay,az,lx,ly,lz,gx,gy,gz,mx,my,mz
0,5.1213,-2.1929,-0.70826,-4.5352,-0.67629,0.081656,0.43616,0.16371,0.19823,-21.9,7.14,5.34
1,5.6116,-2.1248,-1.26670,-4.0408,-0.55974,-0.523690,0.32284,0.43127,0.21197,-21.6,7.50,5.64


In [85]:
#y_train = pd.get_dummies(train_data.label)
y_train = train_data.label
y_train.head(3)

0    walking
1    walking
2    walking
Name: label, dtype: object

## Decision Tree Technique

In [158]:
# more info about decision tree classifier: http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
cross_val_score(clf, X_train, y_train, cv=10)
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=0, splitter='best')

## KNN Technique

In [87]:
# more info about knn: http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=7)
neigh.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='uniform')

## Naive Bayes Technique

In [88]:
# more info about gaussiannb: http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB
from sklearn.naive_bayes import BernoulliNB
clf = GaussianNB()
clf.fit(X_train, y_train)

GaussianNB(priors=None)

# Testing The Model With Signals Collected From The Wrist

In [172]:
## Load Testing Data From Another Participant

test_data = pd.read_csv('../pre-processing/valid-dataset/Participant_10_wrist.csv')
## Randomly Select 
df = pd.DataFrame(test_data)
# Randomly sample 70% of your dataframe
test_data= df.sample(frac=0.1)
X_test = test_data.drop('label', axis = 1)
y_test = test_data.label;#pd.get_dummies(test_data.label)

In [173]:
## Testing Different Models

y_pred = rf.predict(X_test)
print("Decision Tree Classifier Performance: ",rf.score(X_test, y_test))
y_pred = clf.predict(X_test)
print("Gaussian Naive Bayes Performance: ", clf.score(X_test, y_test))
y_pred = neigh.predict(X_test)
print("KNN Performance: ", neigh.score(X_test, y_test))

Decision Tree Classifier Performance:  0.332380952381
Gaussian Naive Bayes Performance:  0.287936507937
KNN Performance:  0.23873015873


# Testing The Model With Signals Collected From The Belt

In [174]:
## Load Testing Data From Another Participant

test_data = pd.read_csv('../pre-processing/valid-dataset/Participant_10_belt.csv')
## Randomly Select 
df = pd.DataFrame(test_data)
# Randomly sample 30% of your dataframe
test_data= df.sample(frac=0.3)
X_test = test_data.drop('label', axis = 1)
y_test = test_data.label;#pd.get_dummies(test_data.label)

In [175]:
## Testing Different Models

y_pred = rf.predict(X_test)
print("Decision Tree Classifier Performance: ",rf.score(X_test, y_test))
y_pred = clf.predict(X_test)
print("Gaussian Naive Bayes Performance: ", clf.score(X_test, y_test))
y_pred = neigh.predict(X_test)
print("KNN Performance: ", neigh.score(X_test, y_test))

Decision Tree Classifier Performance:  0.504021164021
Gaussian Naive Bayes Performance:  0.490317460317
KNN Performance:  0.499841269841
